# Yellow Taxi - Создание интерактивного демо

In [1]:
import numpy as np
import pandas as pd
import holoviews as hv
import hvplot.pandas  # noqa
from bokeh.models import HoverTool, CrosshairTool
hv.extension('bokeh')

import folium
from folium.features import GeoJsonTooltip

# from geojson import Feature, Polygon, FeatureCollection

import geopandas as gpd
import datetime as dt


ModuleNotFoundError: No module named 'holoviews'

In [ ]:
import datetime as dt
import panel as pn
from panel.interact import interact, interactive, fixed, interact_manual
pn.extension()

Загрузим данные с точными и спрогнозированными значениями на июнь

In [ ]:
june_forecast =pd.read_csv('june_forecast.csv', index_col=0)
june_forecast['datetime'] = pd.to_datetime(june_forecast['datetime'])
june_forecast

Интерактив добавим с помощью пакета panel.

In [ ]:
regions_geometry = gpd.read_file('regions_geometry.geojson')
regions_geometry

In [ ]:
regions = np.unique(june_forecast['region'])
regions

In [ ]:
def get_data(date = dt.datetime(2016,6,15,13), pred_hour = 1, df_data = june_forecast, df_geometry = regions_geometry):
    tmp = df_data[df_data['datetime'] == date]
    
    if pred_hour == 0:
        col = 'y_T'
    else:
        col = f'y_P{pred_hour}'
        
    tmp = tmp[['region', col] ]
    res = df_geometry.merge(tmp, right_on='region', left_on='region')    
    res = res.rename(columns={col: 'count'})
    
    res['count'] = np.floor(res['count']).astype(int)
    res['count_m'] = np.cbrt(res['count'] )
    
    return res

# get_data()
res = get_data(pred_hour=1)
res

In [ ]:
def plot_map(date, curr_hour, pred_hour):
    
    c_lat, c_long = -73.89446, 40.74357
    
    curr_datetime = dt.datetime.combine( date, dt.time(curr_hour) )

    geo_data = get_data(curr_datetime, pred_hour)

    m = folium.Map(location=[c_long, c_lat], zoom_start=11, tiles='OpenStreetMap', 
                    max_start=12)  


    cm = folium.Choropleth(
        geo_data=geo_data,
        data=geo_data,
        columns=['region', 'count_m'],
        key_on='feature.properties.region',
        fill_color='YlGnBu', 
        fill_opacity=0.5, 
        line_opacity=1,
        legend_name='Total amount of trips per region',
        smooth_factor=0).add_to(m)
    
    for key in cm._children:
        if key.startswith('color_map'):
            del(cm._children[key])

            
    tooltip = GeoJsonTooltip(
    fields=['region', 'count'],
    aliases=['Регион', 'Поездки'],
    ).add_to(cm.geojson)
            
    folium_pane = pn.pane.plot.Folium(m, height=450)
    
    return folium_pane
    

In [ ]:
date_picker = pn.widgets.DatePicker(name='Укажите день', 
                                            start = dt.date(2016, 6, 1),
                                            end = dt.date(2016, 6, 30),
                                            value=dt.date(2016, 6, 15))

curr_hour_select = pn.widgets.Select(value=20, options=list(range(24)), name='Укажите текущий час')
pred_hour_select = pn.widgets.Select(value=1, options=list(range(1,7)), name='Укажите шаг прогноза')

row1 = pn.Row(date_picker, curr_hour_select, pred_hour_select)
row2 = pn.Row(plot_map(date_picker.value, curr_hour_select.value, 0),
                 plot_map(date_picker.value,curr_hour_select.value, pred_hour_select.value))

def update_day(event):    
    if date_picker.value.day == 30:        
        tmp = row1[1].value
        row1[1].options = list(range(18))
        row1[1].value = min(tmp, 17)
    else:
        row1[1].options = list(range(24))
        row2[0] = plot_map(date_picker.value, curr_hour_select.value,0)
 
def update_curr_hour(event):
    row2[0] = plot_map(date_picker.value, curr_hour_select.value, 0)
    
def update_pred_hour(event):
    row2[1] = plot_map(date_picker.value, curr_hour_select.value, pred_hour_select.value)


layout =pn.Column(row1, row2)



date_picker.param.watch(update_day, 'value')
curr_hour_select.param.watch(update_curr_hour, 'value')
pred_hour_select.param.watch(update_pred_hour, 'value')

layout


In [ ]:
def xtick_formatter(value):
    return str(value) + ':00'

def plot_ts4(region, day,  hour ,  data = june_forecast):    
    tmp = data[(data['region'] == region) & (data['datetime'].dt.day == day) & (data['datetime'].dt.hour == hour)]
    
    df_true = pd.DataFrame()
    df_predict = pd.DataFrame()
    
    df_true['hour'] = np.linspace(1,6,6) + hour
    df_true['true'] = tmp[['y_T1','y_T2','y_T3','y_T4','y_T5','y_T6']].values[0]
    df_predict['hour'] = np.linspace(1,6,6) + hour
    df_predict['predict'] = np.floor(tmp[['y_P1','y_P2','y_P3','y_P4','y_P5','y_P6']].values[0])
    
    
    tooltips_true = [("Поездки", "@true")]    
    hover_true = HoverTool(tooltips=tooltips_true, mode='mouse')
    
    tooltips_predict = [("Поездки", "@predict")]    
    hover_predict = HoverTool(tooltips=tooltips_predict, mode='mouse')
    
    curve1 = hv.Curve((df_true['hour'], df_true['true']), label='Истинные значения').opts()
    dots1 = hv.Points(df_true).opts(color='blue', size=9, marker='o', tools=[hover_true])
    
    
    curve2 = hv.Curve((df_predict['hour'], df_predict['predict']), label='Предсказанные занчения')#.opts(color='green')
    dots2 = hv.Scatter( df_predict).opts(color='red', size=9, marker='o', tools=[hover_predict])
    
    ov = hv.Overlay([curve1,dots1, curve2,dots2]).opts(height=400,  width=600,
    xlabel='Время', ylabel='Количество поездкок',  title= f'Прогноз числа поездок из региона {region}',
    legend_position='top',
    xlim = (hour, hour +7), xticks = df_predict['hour'], xformatter=xtick_formatter  )
    
    fig = pn.panel(ov)
    
    return fig

In [ ]:
region_select = pn.widgets.Select(name='Выберите регион', options=list(regions), size=1 )

date_picker2 = pn.widgets.DatePicker(name='Укажите день', 
                                            start = dt.date(2016, 6, 1),
                                            end = dt.date(2016, 6, 30),
                                            value=dt.date(2016, 6, 15))

hour_select = pn.widgets.Select(value=10, options=list(range(24)), name='Укажите час')

col1 = pn.Column( region_select, date_picker2, hour_select)
col2 = plot_ts4(region_select.value, date_picker2.value.day, hour_select.value)

layout2 =pn.Row(col1, col2)



def update(event):       
    if date_picker2.value.day == 30:
        tmp = hour_select.value
        hour_select.options = list(range(18))
        hour_select.value = min(tmp, 17)
    else:
        hour_select.options = list(range(24))
        
    layout2[1] = plot_ts4(region_select.value, date_picker2.value.day, hour_select.value)
 
    
    

region_select.param.watch(update, 'value')
date_picker2.param.watch(update, 'value')
hour_select.param.watch(update, 'value')

layout2

In [ ]:
layout3 = pn.Tabs(('Map', layout), ('Time Series', layout2))

layout3

Вкладка Maps: 

    Слева отображается карта, на которой регионы окрашены в цвета согласно числу поездок на "текущий час".
    
    Справа отображается карта, на которой регионы окрашены в цвета согласно числу поездок на "шаг прогноза".

Вкладка Time Series: 

    Слева указываются регион, дата и час.
    
    Справа выводится график истинных и прогнозных значений на 6 часов вперед
